# 1. Loading libraries and data

In [1]:
import matplotlib.pyplot as plt 
%matplotlib qt
import numpy as np
import pyxem as pxm
import hyperspy.api as hs
from pyxem.libraries.calibration_library import CalibrationDataLibrary
from pyxem.generators.calibration_generator import CalibrationGenerator
#matplotlib inline

In [2]:
s1=hs.load('STEM SI.dm4')

In [3]:
dp=s1[2]

In [4]:
dp.set_signal_type('electron_diffraction')

In [5]:
s1[2].plot()

# 2. Calibration of NBED pattern

In [6]:
import logging

# Set up logging
"""
Instead of using print statements, using Python’s built-in logging module. 
This allows for better control over the output and can be easily toggled or configured.
"""
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class NBEDCalibration:
    """
    Class to calibrate NBED patterns based on microscope metadata.

    Parameters:
        NBED_data (Signal): Hyperspy signal containing NBED data.
        microscope_model (str, optional): Type of microscope (e.g., 'F30' or 'HD2700').
                                         If not provided, it will be read from metadata.
    """
    
    def __init__(self, NBED_data: hs.signals.Signal2D, microscope_model: str = None):
        self.s = NBED_data
        self.microscope_model = microscope_model
        self.nbed_units = None
        
        # Calibration dictionaries for F30 at paper building and HD2700 at Marcus building, will add more if we have new TEM/STEM
        self._calibration_dicts = {
            'F30': {
                200: 0.2345746633560363,
                250: 0.1862625415225771,
                300: 0.1562169503514413,
                380: 0.1225684262753367,
                580: 0.0829012794983478,
                750: 0.0621134215923645,
                1000: 0.0463765534986008,
                1200: 0.0385675398614808,
                1500: 0.0304885392056699,
                2000: 0.022962175785559,
                3000: 0.0153784852837892,
                4500: 0.0101846964140601,
            },
            'HD2700': {
                1: 0.268622,
                1.049: 0.214803,
                1.1: 0.175866,
                1.149: 0.150919,
                1.2: 0.129091,
                1.251: 0.11219,
                1.3: 0.09901,
                1.351: 0.089018,
                1.4: 0.079783,
                1.451: 0.072484,
                1.5: 0.066695,
                1.551: 0.061213,
                1.6: 0.056804,
                1.651: 0.052335,
                1.699: 0.048822,
                1.751: 0.045686,
                1.8: 0.04286,
                1.851: 0.040391,
                1.881: 0.039397,
                1.951: 0.03637,
                2: 0.03413,
            }
        }
    
    def calibration(self) -> hs.signals.Signal2D:
        """
        Calibrate the NBED signal based on camera length and microscope type.

        Returns:
            Signal2D: Calibrated hyperspy signal.
        """
        try:
            # Determine microscope model from metadata if not provided
            if self.microscope_model is None:
                if hasattr(self.s.metadata.Acquisition_instrument.TEM, 'microscope'):
                    self.microscope_model = self.s.metadata.Acquisition_instrument.TEM.microscope
                else:
                    self.microscope_model = 'F30'
            
            logger.info(f'Microscope model: {self.microscope_model}')
            
            # Retrieve camera length from metadata
            camera_length = self.s.metadata.Acquisition_instrument.TEM.camera_length
            logger.info(f'Camera length = {camera_length} mm')
            
            # Adjust original scale based on current units
            original_offset = self.s.axes_manager[2].offset
            axis0_units = self.s.axes_manager[0].units
            if axis0_units == 'µm':
                original_scale = self.s.axes_manager[0].scale
            elif axis0_units == 'nm': #handle in case nm
                original_scale = self.s.axes_manager[0].scale / 1000
            else:
                raise ValueError(f"Unexpected unit '{axis0_units}' for axis 0")
            
            # Look up calibration value using a helper function
            self.nbed_units = self._lookup_calibration(camera_length, self.microscope_model)
            logger.info(f'Calibration units: {self.nbed_units}')
            
            # Apply calibration to axes
            self._apply_calibration(original_scale, original_offset)
            
        except AttributeError as err:
            logger.warning("Missing metadata. Switching to manual input for camera length.")
            try:
                camera_length = float(input("Please enter camera length (CL) for HD2700: "))
                logger.info(f'User entered camera length: {camera_length}')
                self.nbed_units = self._lookup_calibration(camera_length, 'HD2700')
                self._apply_calibration(original_scale, original_offset)
            except Exception as inner_err:
                logger.error("Calibration failed: Camera length out of calibrated range or invalid input.")
                raise inner_err
        return self.s
    
    def _lookup_calibration(self, camera_length: float, model: str, tol: float = 0.01) -> float:
        """
        Helper method to look up the calibration value given a camera length and microscope model.
        
        Parameters:
            camera_length (float): The camera length in mm.
            model (str): The microscope model ('F30' or 'HD2700').
            tol (float): Tolerance for matching camera length (used for float comparisons).
        
        Returns:
            float: The calibration value.
        
        Raises:
            ValueError: If no matching calibration value is found.
        """
        calib_dict = self._calibration_dicts.get(model)
        if calib_dict is None:
            raise ValueError(f"Unknown microscope model: {model}")
        
        for key, value in calib_dict.items():
            # For F30, assume keys are integers so cast camera_length for exact matching
            if model == 'F30':
                if int(camera_length) == key:
                    return value
            else:  # For HD2700, use tolerance-based matching
                if abs(camera_length - key) < tol:
                    return value
        raise ValueError(f"No calibration found for camera length {camera_length} in model {model}")
    
    def _apply_calibration(self, original_scale: float, original_offset: float) -> None:
        """
        Applies the calibration to the signal axes.
        
        Parameters:
            original_scale (float): The original scale from axis 0.
            original_offset (float): The original offset from axis 2.
        """
        # Update axis 0 and 1 to nm units (scale factor: µm to nm)
        self.s.axes_manager[0].units = 'nm'
        self.s.axes_manager[0].scale = original_scale * 1000
        self.s.axes_manager[1].units = 'nm'
        self.s.axes_manager[1].scale = original_scale * 1000
        
        # Update diffraction axes (2 and 3) using the calibration factor
        for ax in [2, 3]:
            self.s.axes_manager[ax].units = '/nm'
            self.s.axes_manager[ax].scale = self.nbed_units
            self.s.axes_manager[ax].offset = original_offset * self.nbed_units
"""
Example use:
dp = hs.load('STEM SI.dm4')[2] something [2] or [-1] is the NBED SI map, check your loaded data
dp.set_signal_type('electron_diffraction')

calib = NBEDCalibration(dp, 'F30')
dp_calibrated = calib.calibration()
dp_calibrated.plot()
"""
calib = NBEDCalibration(dp, 'F30')
dp_calibrated = calib.calibration()


In [7]:
calib.nbed_units

0.0463765534986008

# 3. Centering the direct beam

In [8]:
dp = dp_calibrated

In [9]:
# check if the scale, offset, units and other parameters are right
dp.axes_manager

Navigation axis name,size,index,offset,scale,units
z,254,0,-0.0,2.034828532487154,nm
y,18,0,-0.0,2.034828532487154,nm
Signal axis name,size,,offset,scale,units
x,512,,-11.872397695641805,0.0463765534986008,/nm
y,512,,-11.872397695641805,0.0463765534986008,/nm


In [10]:
dp.plot(gamma =0.2, norm = 'power') # gamma =0.2, norm = 'power' help us to visualize the patterns more clearly

In [434]:
dp_shifts = dp.get_direct_beam_position(method="blur", sigma=3, half_square_width=20)
dp_linear_plane = dp_shifts.get_linear_plane()

[########################################] | 100% Completed | 1.83 ss


In [433]:
# Center beam correction method 1: manually shift by passing dp_linear_plane
dp = dp.center_direct_beam(shifts=dp_linear_plane, inplace=False)

[########################################] | 100% Completed | 8.25 sms


In [421]:
dp.plot(gamma =0.2, norm = 'power')

In [407]:
# Read the positions of direct beam
center_shift_value= dp.get_direct_beam_position(method='center_of_mass')

In [439]:
center_shift=-dp_shifts*calib.nbed_units

In [408]:
center_shift_value.inav[38,12]

<BeamShift, title: Diffraction SI, dimensions: (|2)>

## Optional: Create training data for your own NBED patterns

In [23]:

import os
from PIL import Image

# --- Parameters for cropping ---
center = (0, 0)
pixel_size = 0.0463765534986008
number_pixel4training = 20
mask = pixel_size * number_pixel4training
# If slicing requires integer indices, you might convert mask to int:
# mask = int(mask)

# --- Extract the training data ---
# Make sure dp.isig supports slicing with these limits.
CNN_training_data = dp.isig[-mask:mask, -mask:mask].data
print("Shape of training data:", np.shape(CNN_training_data))
dp.isig[-mask:mask, -mask:mask].plot()

# --- Define the output directory ---
output_dir = "training_images"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# --- Helper function to process and save an image ---
def process_and_save_image(y, x, img_array):
    # Normalize the image array to the range [0, 1] based on its min and max.
    img_min = img_array.min()
    img_max = img_array.max()
    if img_max - img_min != 0:
        img_norm = (img_array - img_min) / (img_max - img_min)
    else:
        img_norm = np.zeros_like(img_array)
    
    # Convert normalized values to uint8 (0-255).
    img_uint8 = (img_norm * 255).astype(np.uint8)
    
    # Create a grayscale PIL image (mode "L" means grayscale).
    img = Image.fromarray(img_uint8, mode="L")
    
    # Define the filename as "(x,y).png".
    # Here we add 1 to y so that row 0 becomes 1, row 10 becomes 11, etc.
    filename = f"({x},{y+1}).png"
    filepath = os.path.join(output_dir, filename)
    img.save(filepath)

# --- Process images: select every 10th row (e.g. row indices 0, 10, 20, ...) ---
for y in range(0, CNN_training_data.shape[0], 10):
    total_cols = CNN_training_data.shape[1]
    # Select every 10th column, ensuring the last column is included.
    cols = list(range(0, total_cols, 10))
    if cols[-1] != total_cols - 1:
        cols.append(total_cols - 1)
    for x in cols:
        img_array = CNN_training_data[y, x, :, :]
        process_and_save_image(y, x, img_array)


Shape of training data: (18, 254, 40, 40)


In [12]:
#Center beam correction method 2: using deep learning model (under development)

[########################################] | 100% Completed | 9.85 ss
[########################################] | 100% Completed | 9.95 s
[########################################] | 100% Completed | 4.83 sms
[########################################] | 100% Completed | 4.91 s


In [19]:
dp.plot() #please move to the ROI in the navigation image

In [104]:
dp.axes_manager

Navigation axis name,size,index,offset,scale,units
z,254,33,-0.0,2.034828532487154,nm
y,18,7,-0.0,2.034828532487154,nm
Signal axis name,size,,offset,scale,units
x,512,,-11.872397695641805,0.0463765534986008,/nm
y,512,,-11.872397695641805,0.0463765534986008,/nm


In [487]:
center_shift.inav[39,11].data

array([-0.18550621, -0.41738898])

# 4. Using diffraction SI image to generate VDF, calculate the interatomic distance and orientation for different layers/domain

In [488]:
import logging
import math
import numpy as np
import pandas as pd
from scipy.ndimage import gaussian_filter, center_of_mass
import hyperspy.api as hs
import matplotlib.pyplot as plt
import pickle


# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class SI_RELFECTION_TOOL:
    """
    Class to store calibration parameters and additional reflection-related data 
    in tunable pandas DataFrames and provide utility functions for reflection analysis.

    There are two DataFrames:
      - df_bulk: stores bulk calibration parameters.
      - df_ID_4_VDF: stores individual diffraction pattern (ID_4_VDF) data.
      
    Notes:
      - The method store_ID_4_VDF now receives four inputs:
            domain (int), SI_image_coord (tuple), name (str), and reflection (tuple).
      - For each domain, multiple records can be stored. However, for a given SI_image_coord and species,
        if a new reflection (converted to pixel units) is within 5 pixels of an existing reflection, it is not added.
      - A dictionary attribute self.domain registers records by domain.
      - The method print_domain_summary prints a pivot table summary for a specified domain.
        The summary table shows:
          - SI_image_coord,
          - direction_beam_position (computed from SI_image_coord via center_shift_value),
          - DP_position,
          - reflection,
          - name (species),
          - reci_distance,
          - plane_norm_vect (unit vector from theta),
          - plane_direction (equal to phi),
          - d_spacing_nm (1/reci_distance).
      - The diffraction image is obtained by slicing:
            self.diffraction_signal.inav[SI_image_coord[1], SI_image_coord[0]].data
    """

    def __init__(self, diffraction_signal=None):
        self.df_bulk = pd.DataFrame(
            columns=[
                'navigation_units', 'navigation_scale', 'navigation_offset',
                'DP_units', 'DP_scale', 'DP_offset',
                'camera_length', 'microscope_model', 'nbed_units',
                'reflection_radius_pixel', 'reflection_radius_physical'
            ]
        )
        self.df_ID_4_VDF = pd.DataFrame(
            columns=['SI_image_coord', 'DP_position', 'species', 'reflection']
        )
        self.domain = {}  # e.g., self.domain[0] is a list of records.
        self.nbed_units = None  # Calibration factor (physical unit per pixel)
        self.diffraction_signal = diffraction_signal  # Hyperspy Signal2D for diffraction patterns.
        self.center_shift_value = None  # Signal containing the direct beam (center shift)
        self.dp_offset = None
        self.navigation_scale = None

    @staticmethod
    def calculate_reci_distance(rx, ry, cx, cy) -> float:
        """Calculate the reciprocal length in pixels based on reflection and direct beam positions."""
        return np.sqrt((rx - cx)**2 + (ry - cy)**2)

    @staticmethod
    def orientation(rx, ry, cx, cy):
        dx = cx -rx
        dy = cy - ry
        theta = math.atan2(dy, dx)
        phi = math.pi/2 + theta
        return theta, phi

    def store_bulk_data(self, signal: hs.signals.Signal2D, center_shift_value: hs.signals.Signal2D,
                        microscope_model: str, nbed_units: float, **kwargs):
        """
        Stores the bulk calibration parameters in the df_bulk DataFrame.
        
        The SI image coordinate is not stored here.
        """
        nav_units = signal.axes_manager[0].units
        nav_scale = signal.axes_manager[0].scale
        nav_offset = getattr(signal.axes_manager[0], 'offset', None)

        dp_units = signal.axes_manager[2].units 
        dp_scale = signal.axes_manager[2].scale 
        dp_offset = getattr(signal.axes_manager[2], 'offset', None)

        try:
            camera_length = signal.metadata.Acquisition_instrument.TEM.camera_length
        except AttributeError:
            camera_length = None

        data = {
            'navigation_units': nav_units,
            'navigation_scale': nav_scale,
            'navigation_offset': nav_offset,
            'DP_units': dp_units,
            'DP_scale': dp_scale,
            'DP_offset': dp_offset,
            'camera_length': camera_length,
            'microscope_model': microscope_model,
            'nbed_units': nbed_units
        }

        if "reflection_radius_pixel" not in kwargs:
            kwargs["reflection_radius_pixel"] = 10
        if "reflection_radius_physical" not in kwargs:
            kwargs["reflection_radius_physical"] = 10 * nbed_units

        data.update(kwargs)
        self.df_bulk = pd.concat([self.df_bulk, pd.DataFrame([data])], ignore_index=True)
        logger.info(f"Stored bulk calibration parameters: {data}")
        self.nbed_units = nbed_units
        self.dp_offset = dp_offset
        self.center_shift_value = center_shift_value
        self.navigation_scale= nav_scale

    def get_direct_beam_position(self, signal: hs.signals.Signal2D, SI_image_coord: tuple, nbed_units: float = None):
        """
        Retrieve the direct beam position (in physical units) using the center_shift_value signal.
        
        The direct beam position is extracted by slicing:
            center_shift_value.inav[SI_image_coord[1], SI_image_coord[0]].data
        """
        if nbed_units is None:
            if self.nbed_units is None:
                raise ValueError("Calibration factor (nbed_units) must be provided or set.")
            nbed_units = self.nbed_units

        if self.center_shift_value is None:
            raise ValueError("Center shift value signal is not set. Run store_bulk_data with a valid center_shift_value.")

        pixel_position = self.center_shift_value.inav[SI_image_coord[0], SI_image_coord[1]].data
        pixel_position = np.asarray(pixel_position).flatten()
        if pixel_position.size < 2:
            raise ValueError("Direct beam pixel position must contain at least 2 elements.")
        pixel_position = pixel_position[:2]
        physical_position = pixel_position
        logger.info(f"Direct beam pixel position (from center_shift_value): {pixel_position}, physical position: {physical_position}")
        return physical_position

    def refine_reflection(self, SI_image_coord: tuple, initial_reflection: tuple, radius: int = 6, sigma: float = None, threshold = None):
        """
        Refine the manually input reflection point using center-of-mass or by finding the center of a binary ellipse.
        
        The diffraction image is extracted by slicing:
            self.diffraction_signal.inav[SI_image_coord[0], SI_image_coord[1]].data
        """
        if self.diffraction_signal is None:
            raise ValueError("Diffraction signal is not set.")
        if self.nbed_units is None:
            raise ValueError("Calibration factor (nbed_units) is not set.")

        ROI_image = self.diffraction_signal.inav[SI_image_coord[0], SI_image_coord[1]]
        pixel_initial_x = int(round(initial_reflection[0] / self.nbed_units))
        pixel_initial_y = int(round(initial_reflection[1] / self.nbed_units))
        x_min = pixel_initial_x - radius
        x_max = pixel_initial_x + radius + 1
        y_min = pixel_initial_y - radius
        y_max = pixel_initial_y + radius + 1

        cropped_data = np.array(ROI_image.isig[y_min:y_max, x_min:x_max].data)
        cropped_data = np.clip(cropped_data, 0, None)

        if threshold is not None:
            if sigma is not None:
                logger.warning("Both sigma and threshold provided; using threshold method to compute center of ellipse.")
            binary_data = np.where(cropped_data > threshold, 1, 0)
            com_y, com_x = center_of_mass(binary_data)
        else:
            if sigma is not None:
                cropped_data = gaussian_filter(cropped_data, sigma=sigma)
                logger.info(f"Applied Gaussian blur with sigma (in pixels) = {sigma} to the cropped sub-image.")
            com_y, com_x = center_of_mass(cropped_data)

        refined_x_pixel = x_min + com_x
        refined_y_pixel = y_min + com_y
        refined_x_phys = refined_x_pixel * self.nbed_units
        refined_y_phys = refined_y_pixel * self.nbed_units
        refined_reflection = (refined_x_phys, refined_y_phys)
        logger.info(f"Initial reflection (physical): {initial_reflection}, refined reflection (physical): {refined_reflection}")
        return refined_reflection

    def store_ID_4_VDF(self, domain: int, SI_image_coord: tuple, name: str, reflection: tuple):
        """
        Stores an individual reflection for VDF analysis.
        """
        if self.nbed_units is None:
            raise ValueError("Calibration factor (nbed_units) is not set. Run store_bulk_data first.")
        if self.diffraction_signal is None:
            raise ValueError("Diffraction signal is not set.")

        DP_position = self.get_direct_beam_position(self.diffraction_signal, SI_image_coord, self.nbed_units)
        # Ensure DP_position is a flattened 1D array.
        DP_position = np.array(DP_position).flatten()
        new_px = np.array(reflection) / self.nbed_units

        if domain not in self.domain:
            self.domain[domain] = []

        too_close = False
        for rec in self.domain[domain]:
            if rec['SI_image_coord'] == SI_image_coord and rec['species'] == name:
                stored_px = np.array(rec['reflection']) / self.nbed_units
                dist = np.linalg.norm(new_px - stored_px)
                if dist <= 5:
                    too_close = True
                    logger.warning(f"New reflection {reflection} (in pixels: {new_px}) is within {dist:.2f} pixels of an existing reflection {rec['reflection']} (in pixels: {stored_px}). Not adding.")
                    break

        if not too_close:
            record = {
                'SI_image_coord': SI_image_coord,
                'DP_position': DP_position,
                'species': name,
                'reflection': reflection
            }
            self.domain[domain].append(record)
            self.df_ID_4_VDF = pd.concat([self.df_ID_4_VDF, pd.DataFrame([record])], ignore_index=True)
            logger.info(f"Added reflection for domain {domain}, SI_image_coord {SI_image_coord}, species '{name}': {reflection}")

    def update_reci_orientation(self):
        """
        Updates each domain record with reciprocal distance and orientation.
        """
        for d in self.domain:
            for i, rec in enumerate(self.domain[d]):
                dp = rec.get('DP_position', None)
                refl = rec.get('reflection', None)
                print(f"Domain {d} record {i}: DP_position = {dp}, reflection = {refl}")
                try:
                    reci_distance = float(self.calculate_reci_distance(refl[0], refl[1], dp[0], dp[1]))
                except Exception as e:
                    print(f"Error computing reci_distance for domain {d} record {i}: {e}")
                    reci_distance = np.nan
                theta, phi = self.orientation(refl[0], refl[1], dp[0], dp[1])
                print(f"Domain {d} record {i}: reci_distance = {reci_distance}, theta = {theta}, phi = {phi}")
                rec['reci_distance'] = reci_distance
                rec['theta'] = theta
                rec['phi'] = phi
                rec['d_spacing_nm'] = 1/reci_distance if reci_distance != 0 else np.inf
                rec['plane_norm_vect'] = (math.cos(theta), math.sin(theta))
                rec['plane_direction'] = phi
        # Rebuild the DataFrame from all domain records.
        all_records = []
        for d in self.domain:
            all_records.extend(self.domain[d])
        self.df_ID_4_VDF = pd.DataFrame(all_records)
        logger.info("Updated domain records with orientation parameters.")
        return

    def print_domain_summary(self, domain: int, group_by: str = 'SI_image_coord'):
        """
        Prints a summary pivot table for the specified domain.
        """
        if domain not in self.domain or len(self.domain[domain]) == 0:
            print(f"No reflections stored for domain {domain}.")
            return

        df_dom = pd.DataFrame(self.domain[domain])

        self.update_reci_orientation()
        df_dom = pd.DataFrame(self.domain[domain])
        
        df_dom['direction_beam_position'] = df_dom['SI_image_coord'].apply(
            lambda x: str(self.get_direct_beam_position(self.center_shift_value, x, self.nbed_units))
        )
        df_dom['SI_image_coord_str'] = df_dom['SI_image_coord'].apply(lambda x: str(x))
        df_dom['reflection_str'] = df_dom['reflection'].apply(lambda x: str(x))
        df_dom = df_dom.rename(columns={'species': 'name'})
        
        if group_by == 'SI_image_coord':
            group_key = 'SI_image_coord_str'
        elif group_by == 'reflection':
            group_key = 'reflection_str'
        elif group_by == 'name':
            group_key = 'name'
        else:
            raise ValueError("group_by must be one of 'SI_image_coord', 'name', or 'reflection'")
        
        cols = ['SI_image_coord', 'direction_beam_position', 'DP_position', 'reflection', 'name',
                'reci_distance', 'plane_norm_vect', 'plane_direction', 'd_spacing_nm']
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 200)
        
        pivot_df = df_dom.pivot_table(index=group_key, values=cols, aggfunc=lambda x: list(x))
        print(pivot_df)
        return pivot_df

    def vdf(self, domain: int, radius: int = 10,filtered = False):
        """
        For each reflection record in the specified domain, compute and plot the VDF (integrated intensity)
        using a circular ROI centered on the reflection (in pixel coordinates) with the given radius (in pixels).
        
        For each reflection:
          - Convert the reflection (in physical units) to pixel coordinates.
          - Create a CircleROI using hs.roi.CircleROI with center (cx, cy) set to the reflection (in pixel coordinates)
            and r equal to radius * self.nbed_units.
          - Compute the integrated intensity VDF using self.diffraction_signal.get_integrated_intensity(roi).
          - Plot the VDF.
          - Store the resulting VDF object in the record under the key 'vdf'.
          - Also store the input radius in the record under the key 'radius'.
        """
        
        if domain not in self.domain or len(self.domain[domain]) == 0:
            logger.info(f"No reflections stored for domain {domain}.")
            return

        for rec in self.domain[domain]:
            # Store the input radius in the record.
            rec['radius'] = radius
            reflection_phys = rec['reflection']
            # Here, we assume that the reflection is already in pixel units (or if not, adjust accordingly).
            # If needed, you could convert using: reflection_px = np.array(reflection_phys) / self.nbed_units
            # For now, we'll assume reflection is in pixel coordinates.
            reflection_px = np.array(reflection_phys)
            if filtered is False:
                try:
                    roi = hs.roi.CircleROI(cx=reflection_px[0], cy=reflection_px[1], r_inner=0, r=radius * self.nbed_units)
                    vdf_obj = self.diffraction_signal.get_integrated_intensity(roi)
                    # Use matplotlib to plot the VDF.
                    plt.figure()
                    plt.imshow(np.array(vdf_obj.data), cmap='gray')
                    plt.title(f"Domain {domain}, species {rec['species']} at reflection {rec['reflection']}")
                    plt.axis('off')
                    plt.show()
                    rec['vdf'] = vdf_obj
                    if 'threshold' not in rec:
                        rec['threshold'] = 0
                except Exception as e:
                    logger.error(f"Error computing VDF for record {rec}: {e}")
            elif filtered:
                try:
                    roi = hs.roi.CircleROI(cx=reflection_px[0], cy=reflection_px[1], r_inner=0, r=radius * self.nbed_units)
                    vdf_obj = self.diffraction_signal.get_integrated_intensity(roi)
                    # Use matplotlib to plot the VDF.
                    vdf_data = vdf_obj.data
                    vdf_data[vdf_data <rec['threshold']] =0
                    vdf_obj.data = vdf_data
                    plt.figure()
                    plt.imshow(np.array(vdf_data), cmap='gray')
                    plt.title(f"Domain {domain}, species {rec['species']} at reflection {rec['reflection']}")
                    plt.axis('off')
                    plt.show()
                except Exception as e:
                    logger.error(f"Error computing VDF for record {rec}: {e}")
                
            
        return
    
    
    def plot_vdf_with_line(self, zoom_factor_length, domain: int, overlay=True, threshold=None, colors=None, sample_step=1):
        """
        For each reflection record in the specified domain that has computed VDF data,
        plot the VDF image (after threshold filtering so that only nonzero intensities remain)
        and overlay parallel lines drawn over the nonzero region.

        The line spacing (in pixels) is computed as:
            zoomed_d_spacing = (zoom_factor_length * d_spacing_nm) / navigation_scale
        The line orientation is given by the record's plane_direction (phi).

        If exactly two records are present, a text file is exported with:
            - d-spacings from name 0,
            - angle in degree from name 0,
            - d-spacings from name 1,
            - angle in degree from name 1,
            - d-spacing mismatch,
            - angle mismatch.
        The file name is derived from the SI_image_coord of the first record.

        Parameters:
            zoom_factor_length (float): Multiplier for d_spacing_nm.
            domain (int): The domain index to process.
            overlay (bool): If True, overlay all images in one figure; if False, plot each individually.
            threshold (float, optional): If provided, values below this intensity are set to zero.
            colors (list, optional): List of colors for each record. If None, defaults to cycling ['r','g','b','c','m','gray','w','y'].
            sample_step (float, optional): Step size (in pixels) for sampling along each line.
        """

        # Helper: sample an infinite line (p0,d) over t and return contiguous segments where mask is True.
        def get_line_segments_in_mask(p0, d, mask, sample_step):
            height, width = mask.shape
            diag = np.sqrt(height**2 + width**2)
            t_vals = np.arange(-diag, diag, sample_step)
            pts = np.array([p0 + t * d for t in t_vals])
            valid = []
            for pt in pts:
                x, y = pt
                if 0 <= x < width and 0 <= y < height:
                    valid.append(pt)
                else:
                    valid.append(None)
            valid = np.array(valid, dtype=object)
            inside = []
            for pt in valid:
                if pt is None:
                    inside.append(False)
                else:
                    x, y = pt
                    xi = int(round(x))
                    yi = int(round(y))
                    if xi < 0 or xi >= width or yi < 0 or yi >= height:
                        inside.append(False)
                    else:
                        inside.append(mask[yi, xi])
            inside = np.array(inside, dtype=bool)
            segments = []
            in_seg = False
            start = None
            for j, val in enumerate(inside):
                if val and not in_seg:
                    in_seg = True
                    start = j
                elif not val and in_seg:
                    end = j - 1
                    pt1 = p0 + t_vals[start] * d
                    pt2 = p0 + t_vals[end] * d
                    segments.append((pt1, pt2))
                    in_seg = False
            if in_seg:
                pt1 = p0 + t_vals[start] * d
                pt2 = p0 + t_vals[-1] * d
                segments.append((pt1, pt2))
            return segments

        if domain not in self.domain or len(self.domain[domain]) == 0:
            logger.info(f"No reflections stored for domain {domain}.")
            return

        default_colors = ['r','g','b','c','m','gray','w','y']
        if colors is None:
            colors = [default_colors[i % len(default_colors)] for i in range(len(self.domain[domain]))]

        plots = []  # List to store tuples: (vdf_data, segments, color)
        for i, rec in enumerate(self.domain[domain]):
            if 'vdf' not in rec:
                logger.warning(f"Record {i} for species {rec.get('species')} has no VDF; skipping.")
                continue
            try:
                vdf_data = np.array(rec['vdf'].data)
                # Use threshold from record if present; else use provided threshold.
                th = rec.get('threshold', threshold)
                if th is not None:
                    vdf_data = np.where(vdf_data < th, 0, vdf_data)
                rec['vdf'].data = vdf_data  # Update the vdf object's data.
                mask = (vdf_data != 0)
            except Exception as e:
                logger.error(f"Error processing VDF for record {i}: {e}")
                continue

            try:
                # Compute the centroid of the nonzero region.
                obj_center = np.array(center_of_mass(mask.astype(float)))
            except Exception as e:
                logger.error(f"Error computing center of mass for record {i}: {e}")
                continue

            if 'd_spacing_nm' not in rec or np.isnan(rec['d_spacing_nm']):
                logger.warning(f"Record {i} missing d_spacing_nm; skipping line overlay.")
                continue

            # Compute zoomed spacing in pixels.
            zoomed_d_spacing = (zoom_factor_length * rec['d_spacing_nm']) / self.navigation_scale

            # Get line orientation from plane_direction.
            phi = rec.get('plane_direction', 0)
            d = np.array([math.cos(phi), math.sin(phi)])
            n = np.array([-math.sin(phi), math.cos(phi)])
            height, width = vdf_data.shape
            # Instead of a fixed bounding box, we use the object center.
            # For simplicity, choose L as a fraction of the image diagonal.
            L = np.sqrt(width**2 + height**2) / 4.0
            offsets = np.arange(-L, L + zoomed_d_spacing, zoomed_d_spacing)
            segments = []
            for off in offsets:
                p0 = obj_center + off * n
                segs = get_line_segments_in_mask(p0, d, mask, sample_step)
                if segs:
                    segments.extend(segs)
            rec['line_segments'] = segments
            plots.append((vdf_data, segments, colors[i]))
        rec0 = self.domain[domain][0]
        rec1 = self.domain[domain][1]
        si_coord_str_0 = str(rec0.get('SI_image_coord')).replace(" ", "_").replace(",", "_")
        si_coord_str_1 = str(rec1.get('SI_image_coord')).replace(" ", "_").replace(",", "_")
        file_name = f"{si_coord_str_0 + si_coord_str_1 }"
        title_str = f"{file_name}  D-spacing are magnified by {zoom_factor_length}"
        # Now, if overlay is True, plot all images together; otherwise, individually.
        if overlay:
            fig, ax = plt.subplots(figsize=(8,8))
            for (vdf_data, segments, col) in plots:
                ax.imshow(vdf_data, cmap='gray', alpha=0.5)
            for (vdf_data, segments, col) in plots:
                for (pt1, pt2) in segments:
                    ax.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color=col, linewidth=1)
            # Convert tick labels from pixels to physical units.
            xticks = ax.get_xticks()
            yticks = ax.get_yticks()
            ax.set_xticklabels([f"{(x * self.navigation_scale):.2f}" for x in xticks])
            ax.set_yticklabels([f"{(y * self.navigation_scale):.2f}" for y in yticks])
            ax.set_title(title_str)
            ax.axis('off')
            plt.tight_layout()
            plt.savefig(title_str )
            plt.show()
        else:
            for (vdf_data, segments, col) in plots:
                fig, ax = plt.subplots(figsize=(8,8))
                ax.imshow(vdf_data, cmap='gray')
                for (pt1, pt2) in segments:
                    ax.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color=col, linewidth=1)
                xticks = ax.get_xticks()
                yticks = ax.get_yticks()
                ax.set_xticklabels([f"{(x * self.navigation_scale):.2f}" for x in xticks])
                ax.set_yticklabels([f"{(y * self.navigation_scale):.2f}" for y in yticks])
                ax.set_title(title_str)
                ax.axis('off')
                plt.tight_layout()
                plt.savefig(title_str )
                plt.show()

        # --- Export a text file if exactly two records exist ---
        if len(self.domain[domain]) == 2:
            rec0 = self.domain[domain][0]
            rec1 = self.domain[domain][1]
            d0 = rec0.get('d_spacing_nm', np.nan)
            d1 = rec1.get('d_spacing_nm', np.nan)
            phi0 = rec0.get('plane_direction', 0)
            phi1 = rec1.get('plane_direction', 0)
            angle0 = math.degrees(phi0)
            angle1 = math.degrees(phi1)
            mismatch_d = abs(d0 - d1)
            mismatch_angle = abs(angle0 - angle1)
            # Create file name from SI_image_coord of the first record.
            si_coord_str_0 = str(rec0.get('SI_image_coord')).replace(" ", "_").replace(",", "_")
            si_coord_str_1 = str(rec1.get('SI_image_coord')).replace(" ", "_").replace(",", "_")
            file_name = f"{si_coord_str_0 + si_coord_str_1 }.txt"
            with open(file_name, "w") as f:
                f.write(f"d-spacings from name 0: {d0:.3f} nm\n")
                f.write(f"angle in degree from name 0: {angle0:.1f}°\n")
                f.write(f"d-spacings from name 1: {d1:.3f} nm\n")
                f.write(f"angle in degree from name 1: {angle1:.1f}°\n")
                f.write(f"d-spacing mismatch between name 0 and name 1: {mismatch_d:.3f} nm\n")
                f.write(f"angle mismatch between name 0 and name 1: {mismatch_angle:.1f}°\n")
            logger.info(f"Exported comparison file: {file_name}")

    def plot_dp(self, domain: int, colors=None, num_col=2, scale_bar_length_nm=1,vmax=0.8,vmin=0.1):
        if domain not in self.domain or len(self.domain[domain]) == 0:
            logger.info(f"No reflections stored for domain {domain}.")
            return

        default_colors = ['r', 'g', 'b', 'c', 'm', 'gray', 'w', 'y']
        if colors is None:
            colors = [default_colors[i % len(default_colors)] for i in range(len(self.domain[domain]))]
        images = []
        DP_list = []  # Direct Beam positions (in physical units)
        RB_list = []  # Reflection positions (in physical units)
        species_list = []
        for rec in self.domain[domain]:
            SI = rec.get('SI_image_coord')
            img = self.diffraction_signal.inav[SI[0], SI[1]].data
            images.append(img)
            dp= rec.get('DP_position')
            dx,dy = dp
            dx-=self.dp_offset
            dy-=self.dp_offset
            DP_list.append((dx/self.nbed_units,dy/self.nbed_units))
            #DP_list.append(dp)
            rp=rec.get('reflection')
            rx,ry = rp
            rx-=self.dp_offset
            ry-=self.dp_offset
            RB_list.append((rx/self.nbed_units,ry/self.nbed_units))
            species_list.append(rec.get('species'))
            #RB_list.append(rp)
        n_images = len(self.domain[domain])
        ncols = num_col
        nrows = int(np.ceil(n_images / ncols))
        fig, axes = plt.subplots(nrows, ncols, figsize=(5, 5))
        if nrows == 1:
            axes = np.expand_dims(axes, axis=0)
        
        # Flatten axes array for easier iteration
        axes = axes.flatten()
        
        for i, img in enumerate(images):
            ax = axes[i]
            maxv = np.max(img)
            minv=np.min(img)
            ax.imshow(img, cmap='gray',vmax=vmax*maxv,vmin=vmin*minv)
            # Retrieve DP and RB (both in physical units).
            DP = DP_list[i]
            RB = RB_list[i]
            # Draw a line from DP to RB.
            ax.plot([DP[0], RB[0]], [DP[1], RB[1]], color=colors[i], linewidth=2)
            # Add a scale bar: convert scale_bar_length_nm to pixels.
            # Here: pixels = (scale_bar_length_nm) / nbed_units.
            scale_bar_length_pixels = scale_bar_length_nm / self.nbed_units
            # Place the scale bar in the lower-left corner.
            # We choose 10 pixels from the left and 10 pixels from the bottom.
            y_bar = img.shape[0] - 10
            x_bar = 10
            
            ax.plot([x_bar, x_bar + scale_bar_length_pixels], [y_bar, y_bar], color='white', linewidth=3)
            ax.text(x_bar, y_bar - 10, f"{scale_bar_length_nm} nm", color='white', fontsize=12)
            # Set the subplot title to the species name.
            ax.set_title(f"{species_list[i]}")
            ax.axis('off')
        
        # Hide any extra subplots.
        for j in range(i + 1, len(axes)):
            axes[j].axis('off')
        
        plt.tight_layout()
        plt.show()
                        
        


    def _sanitize(self, obj):
        """
        Recursively sanitize the object so that it contains only pickleable types.
        For any non-basic type (including re.Match objects), return its string representation.
        """
        import re
        # If the object is a re.Match, return its matched string.
        if isinstance(obj, re.Match):
            try:
                return obj.group(0)
            except Exception:
                return str(obj)
        # If the object is of a basic type, return it.
        elif isinstance(obj, (int, float, str, bool, type(None))):
            return obj
        # If it's a list or tuple, sanitize each element.
        elif isinstance(obj, list):
            return [self._sanitize(item) for item in obj]
        elif isinstance(obj, tuple):
            return tuple(self._sanitize(item) for item in obj)
        # If it's a dictionary, sanitize keys and values.
        elif isinstance(obj, dict):
            new_dict = {}
            for k, v in obj.items():
                new_key = self._sanitize(k)
                new_dict[new_key] = self._sanitize(v)
            return new_dict
        else:
            # Fallback: return the string representation.
            return str(obj)

    def save_domain(self, filename):
        """
        Saves the current self.domain dictionary to a file.
        This method sanitizes self.domain (removing or converting non-pickleable objects)
        before saving, using dill if available.
        
        Parameters:
            filename (str): The file path to save the domain data.
        """
        try:
            import dill as pickle
        except ImportError:
            import pickle
            logger.warning("dill not installed; using pickle, which may fail on unpickleable objects.")
        try:
            sanitized_domain = self._sanitize(self.domain)
            with open(filename, "wb") as f:
                pickle.dump(sanitized_domain, f)
            logger.info(f"Saved domain data to {filename}")
        except Exception as e:
            logger.error(f"Error saving domain data: {e}")

    def load_domain(self, filename):
        """
        Loads domain data from a file (previously saved using save_domain)
        and assigns it to self.domain.
        
        Parameters:
            filename (str): The file path from which to load the domain data.
        """
        try:
            import dill as pickle
        except ImportError:
            import pickle
            logger.warning("dill not installed; using pickle.")
        try:
            with open(filename, "rb") as f:
                self.domain = pickle.load(f)
            logger.info(f"Loaded domain data from {filename}")
        except Exception as e:
            logger.error(f"Error loading domain data: {e}")





# Example Usage:
tool = SI_RELFECTION_TOOL(diffraction_signal=dp)
tool.store_bulk_data(dp, center_shift_value = center_shift, microscope_model=calib.microscope_model, nbed_units=calib.nbed_units)
# tool.store_ID_4_VDF(domain=0, SI_image_coord=(136, 11), name="HZO", reflection=refined_refl_HZO)
# tool.store_ID_4_VDF(domain=0, SI_image_coord=(37, 14), name="WO3", reflection=(-0.9420005826579861, -2.245628707977825))
# tool.update_reci_orientation()
# tool.print_domain_summary(domain=0, group_by='SI_image_coord')
# tool.vdf(domain=0, radius=10)


C:\Users\mengk\AppData\Local\Temp\ipykernel_18972\3705907802.py:115: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df_bulk = pd.concat([self.df_bulk, pd.DataFrame([data])], ignore_index=True)


In [489]:
#debug only
tool.store_ID_4_VDF(domain=0,SI_image_coord=(37,9),reflection=(1.1848122244606938, -3.4141739238528555), name='HZO')
tool.print_domain_summary(domain=0)
tool.store_ID_4_VDF(domain=0,SI_image_coord=(37,14),reflection=(-0.9420005826579861, -2.245628707977825), name='WO3')
tool.print_domain_summary(domain=0)
tool.vdf(domain=0, radius=8)
tool.domain[0][0]['threshold']=200000
tool.domain[0][1]['threshold']=200000
tool.save_domain("(37 9)(37 14).pkl")

Domain 0 record 0: DP_position = [-0.23188277 -0.46376553], reflection = (1.1848122244606938, -3.4141739238528555)
Domain 0 record 0: reci_distance = 3.272909158733642, theta = 2.018453741978744, phi = 3.5892500687736404
                                                       DP_position SI_image_coord           d_spacing_nm      direction_beam_position   name       plane_direction  \
SI_image_coord_str                                                                                                                                                   
(37, 9)             [[-0.23188276749300402, -0.46376553498600803]]      [(37, 9)]  [0.30553857485825275]  [[-0.23188277 -0.46376553]]  [HZO]  [3.5892500687736404]   

                                                plane_norm_vect        reci_distance                                   reflection  
SI_image_coord_str                                                                                                                 
(37, 9)         

In [481]:
center_shift.inav[37,9]

<BeamShift, title: Diffraction SI, dimensions: (|2)>

In [475]:
tool.plot_dp(domain=0, num_col=2, scale_bar_length_nm=1,vmin=0.2,vmax=0.03)

In [484]:
center_shift.inav[37,9]

<BeamShift, title: Diffraction SI, dimensions: (|2)>

In [443]:
tool.plot_vdf_with_line(zoom_factor_length=5, domain =0, overlay=True)

C:\Users\mengk\AppData\Local\Temp\ipykernel_18972\1071083515.py:503: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels([f"{(x * self.navigation_scale):.2f}" for x in xticks])
C:\Users\mengk\AppData\Local\Temp\ipykernel_18972\1071083515.py:504: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yticklabels([f"{(y * self.navigation_scale):.2f}" for y in yticks])


In [514]:
tool.diffraction_signal.plot()

In [221]:
dp.plot(gamma =0.2, norm = 'power')

In [222]:
roi = hs.roi.CircleROI(cx=0,cy=0, r_inner=0, r=10*calib.nbed_units) # mask on specific reflection point

In [223]:
roi.gui() # after run this code, drag the ROI to the flection point

In [234]:
dp.plot_integrated_intensity(roi)

In [348]:
SI_image_coord=(37,9) # your input  # (x,y):the coordinations of picked spot in the SI image. Read from Diffraction SI Signal
HZO_coord=(roi.cx,roi.cy)

In [110]:
refined_refl_HZO = tool.refine_reflection(SI_image_coord=(37,9), initial_reflection=(1.1848122244606938, -3.4141739238528555), radius=10)


In [139]:
# this is the cooridation need to be stored
tool.store_ID_4_VDF(domain=0,SI_image_coord=(37,9),reflection=(1.1848122244606938, -3.4141739238528555), name='HZO')
tool.print_domain_summary(domain=0)

Domain 0 record 0: DP_position = [-0.23117293 -0.40162881], reflection = (1.1848122244606938, -3.4141739238528555)
Domain 0 record 0: reci_distance = 3.328729792809706, theta = -1.131411256161757, phi = 2.7022075829566536
                                                      DP_position SI_image_coord           d_spacing_nm      direction_beam_position   name       plane_direction  \
SI_image_coord_str                                                                                                                                                  
(37, 9)             [[-0.23117292821232427, -0.4016288070208239]]      [(37, 9)]  [0.30041489163826735]  [[-0.23117293 -0.40162881]]  [HZO]  [2.7022075829566536]   

                                                 plane_norm_vect        reci_distance                                   reflection  
SI_image_coord_str                                                                                                                  
(37, 9)         

,DP_position,SI_image_coord,d_spacing_nm,direction_beam_position,name,plane_direction,plane_norm_vect,reci_distance,reflection
SI_image_coord_str,,,,,,,,,
"(37, 9)","[[-0.23117292821232427, -0.4016288070208239]]","[(37, 9)]",[0.30041489163826735],[[-0.23117293 -0.40162881]],[HZO],[2.7022075829566536],"[(0.42538302620166013, -0.9050134148284863)]",[3.328729792809706],"[(1.1848122244606938, -3.4141739238528555)]"


In [415]:
SI_image_coord_WO3=(37,14)
WO3_coord=(roi.cx,roi.cy)

In [438]:
refined_refl_WO3 = tool.refine_reflection(SI_image_coord_WO3, initial_reflection=WO3_coord, radius=10)
refined_refl_WO3 

(-0.9420005826579861, -2.245628707977825)

In [140]:
tool.store_ID_4_VDF(domain=0,SI_image_coord=(37,14),reflection=(-0.9420005826579861, -2.245628707977825), name='WO3')
tool.print_domain_summary(domain=0)

Domain 0 record 0: DP_position = [-0.23117293 -0.40162881], reflection = (1.1848122244606938, -3.4141739238528555)
Domain 0 record 0: reci_distance = 3.328729792809706, theta = -1.131411256161757, phi = 2.7022075829566536
Domain 0 record 1: DP_position = [ 0.01408687 -0.04770609], reflection = (-0.9420005826579861, -2.245628707977825)
Domain 0 record 1: reci_distance = 2.3968660883624398, theta = -1.9811030853315665, phi = 3.5518994121264633
                                                      DP_position SI_image_coord           d_spacing_nm      direction_beam_position   name       plane_direction  \
SI_image_coord_str                                                                                                                                                  
(37, 14)            [[0.01408687459149963, -0.04770609340922435]]     [(37, 14)]  [0.41721145993734216]  [[ 0.01408687 -0.04770609]]  [WO3]  [3.5518994121264633]   
(37, 9)             [[-0.23117292821232427, -0.401628807020

,DP_position,SI_image_coord,d_spacing_nm,direction_beam_position,name,plane_direction,plane_norm_vect,reci_distance,reflection
SI_image_coord_str,,,,,,,,,
"(37, 14)","[[0.01408687459149963, -0.04770609340922435]]","[(37, 14)]",[0.41721145993734216],[[ 0.01408687 -0.04770609]],[WO3],[3.5518994121264633],"[(-0.3988906438668392, -0.9169985028534662)]",[2.3968660883624398],"[(-0.9420005826579861, -2.245628707977825)]"
"(37, 9)","[[-0.23117292821232427, -0.4016288070208239]]","[(37, 9)]",[0.30041489163826735],[[-0.23117293 -0.40162881]],[HZO],[2.7022075829566536],"[(0.42538302620166013, -0.9050134148284863)]",[3.328729792809706],"[(1.1848122244606938, -3.4141739238528555)]"


In [141]:
tool.vdf(domain=0, radius=10)

In [116]:
tool.domain[0]

[{'SI_image_coord': (37, 14),
  'DP_position': array([ 0.01408687, -0.04770609]),
  'species': 'WO3',
  'reflection': (-0.9420005826579861, -2.245628707977825),
  'reci_distance': 2.3968660883624398,
  'theta': -1.9811030853315665,
  'phi': 3.5518994121264633,
  'd_spacing_nm': 0.41721145993734216,
  'plane_norm_vect': (-0.3988906438668392, -0.9169985028534662),
  'plane_direction': 3.5518994121264633,
  'radius': 10,
  'vdf': <Signal2D, title: Integrated intensity, dimensions: (|254, 18)>,
  'threshold': 0},
 {'SI_image_coord': (37, 9),
  'DP_position': array([-0.23117293, -0.40162881]),
  'species': 'HZO',
  'reflection': (1.1848122244606938, -3.4141739238528555),
  'reci_distance': 3.328729792809706,
  'theta': -1.131411256161757,
  'phi': 2.7022075829566536,
  'd_spacing_nm': 0.30041489163826735,
  'plane_norm_vect': (0.42538302620166013, -0.9050134148284863),
  'plane_direction': 2.7022075829566536,
  'radius': 10,
  'vdf': <Signal2D, title: Integrated intensity, dimensions: (|254

In [142]:
tool.domain[0][1]['threshold']=10000
tool.domain[0][0]['threshold']=15000

In [147]:
np.max(tool.domain[0][0]['vdf'].data)

1105334.8

In [143]:
tool.plot_vdf_with_line(zoom_factor_length=10, domain =0, overlay=True)

In [122]:
dp.inav[37,9].isig[(roi.cx-0.2):(roi.cx+0.2),(roi.cy-0.2):(roi.cy+0.2)].plot()

In [61]:
roi2 = hs.roi.CircleROI(cx=Center_of_reflection[0],cy=Center_of_reflection[1], r_inner=0, r=radius) # this is the same as roi, but for selecting VDF image

In [68]:
np.shape(center_shift_value.data)

(18, 254, 2)

In [75]:
import math
reflection_x = roi2.cx
reflection_y = roi2.cy
center_x = center_shift_value.data[SI_image_coord[1],SI_image_coord[0],0]
center_y = center_shift_value.data[SI_image_coord[1],SI_image_coord[0],1]
def calculate_reci_distance(rx,ry,cx,cy)->int:
    """
    Param rx: the x coordinate of a reflection point
    Param ry: the y coordinate of a reflection point
    Param cx: the x coordinate of the direct beam
    Param cy: the y coordinate of the direct beam

    Returns:
    the reciprocal length in pixel
    """
    return np.sqrt((rx-cx)**2 + (ry-cx)**2) # length of g vector /nm
d = calculate_reci_distance(reflection_x,reflection_y,center_x,center_y)
def orientation(rx,ry,cx,cy):
    dx = rx - cx
    dy = ry - cy
    theta = math.atan2(dy,dx) # this is the orientation of the incident g vector
    phi = math.pi/2 - theta
    return theta, phi
theta,phi = orientation(reflection_x,reflection_y,center_x,center_y)
angle = (math.pi-phi)%math.pi#*360/math.pi


In [544]:
vdf = dp.get_integrated_intensity(roi)
vdf.plot()
vdf_data=vdf.data.data
vdf_pixel_to_nm=dp.axes_manager[1].scale
DP_pixel_to_nm=dp.axes_manager[3].scale

In [546]:
#intensity based filtering particles
vdf_data=vdf.data
vdf_data[vdf_data<130000]=0
vdf_data

AttributeError: 'numpy.ndarray' object has no attribute 'plot'

In [547]:
def get_line_segment(p0, d, xmin, xmax, ymin, ymax):
    """
    Given an infinite line (through point p0 with direction d) and a rectangle
    defined by [xmin, xmax] and [ymin, ymax], compute the two intersection
    points of the line with the rectangle boundaries.
    """
    intersections = []
    
    # Intersect with vertical boundaries (x = xmin and x = xmax)
    if d[0] != 0:
        t = (xmin - p0[0]) / d[0]
        y = p0[1] + t * d[1]
        if ymin <= y <= ymax:
            intersections.append((xmin, y))
        t = (xmax - p0[0]) / d[0]
        y = p0[1] + t * d[1]
        if ymin <= y <= ymax:
            intersections.append((xmax, y))
    
    # Intersect with horizontal boundaries (y = ymin and y = ymax)
    if d[1] != 0:
        t = (ymin - p0[1]) / d[1]
        x = p0[0] + t * d[0]
        if xmin <= x <= xmax:
            intersections.append((x, ymin))
        t = (ymax - p0[1]) / d[1]
        x = p0[0] + t * d[0]
        if xmin <= x <= xmax:
            intersections.append((x, ymax))
    
    # Remove duplicates
    intersections = list(set(intersections))
    
    if len(intersections) < 2:
        return None
    elif len(intersections) > 2:
        # If there are more than two, choose the two farthest apart.
        dists = {}
        for i in range(len(intersections)):
            for j in range(i + 1, len(intersections)):
                pt1 = np.array(intersections[i])
                pt2 = np.array(intersections[j])
                dists[(i, j)] = np.linalg.norm(pt1 - pt2)
        i_max, j_max = max(dists, key=dists.get)
        return intersections[i_max], intersections[j_max]
    else:
        return intersections[0], intersections[1]

In [549]:
from scipy.ndimage import label, find_objects
angle =30
mask = (vdf_data != 0)

# Label the connected regions (using 8-connectivity)
structure = np.ones((3, 3), dtype=int)  # This defines connectivity; 8-connectivity for a 2D array
labeled_array, num_features = label(mask, structure=structure)

print("Number of connected non-zero regions:", num_features)

# Get bounding boxes (slices) for each region
slices = find_objects(labeled_array)

d = np.array([np.cos(angle), np.sin(angle)])
# Normal (perpendicular) direction (used to offset the lines):
n = np.array([-np.sin(angle), np.cos(angle)])
# Spacing between parallel lines (in pixels)
spacing = 0.5

for i, sl in enumerate(slices, start=1):
    # Extract the region from the original data using its bounding box
    region = vdf_data[sl]
    
    # Create a new figure for each region
    plt.figure()
    plt.title(f"Region {i} (with parallel lines, angle = {angle*360/math.pi}°)")
    
    # Display the region image
    # Note: imshow uses (row, col) coordinates; rows correspond to y and cols to x.
    plt.imshow(region, cmap='gray', origin='upper')
    
    # Define the coordinate system for the region slice
    # x corresponds to columns (width) and y to rows (height)
    height, width = region.shape
    xmin, xmax = 0, width
    ymin, ymax = 0, height
    
    # Calculate the center of the bounding box
    center = np.array([(xmin + xmax) / 2, (ymin + ymax) / 2])
    
    # Determine the range of offsets needed to cover the rectangle along the normal direction.
    # Get projections of the rectangle corners along the normal
    corners = np.array([
        [xmin, ymin],
        [xmax, ymin],
        [xmax, ymax],
        [xmin, ymax]
    ])
    offsets = np.dot(corners - center, n)
    min_offset = np.min(offsets)
    max_offset = np.max(offsets)
    
    # Create offsets from (min_offset - spacing) to (max_offset + spacing)
    offset_values = np.arange(min_offset - spacing, max_offset + spacing, spacing)
    
    # For each offset, compute the line segment within the rectangle and plot it
    for offset in offset_values:
        p0 = center + offset * n  # starting point for the infinite line, offset from center
        seg = get_line_segment(p0, d, xmin, xmax, ymin, ymax)
        if seg is not None:
            (x1, y1), (x2, y2) = seg
            plt.plot([x1, x2], [y1, y2], color='red', linewidth=1)
    
    # Adjust plot settings
    plt.xlim(xmin, xmax)
    plt.ylim(ymax, ymin)  # invert y-axis to match image coordinate system (origin at top-left)
    plt.xlabel("X (pixels)")
    plt.ylabel("Y (pixels)")
    plt.show()

Number of connected non-zero regions: 7


In [550]:
def get_line_segment(p0, d, xmin, xmax, ymin, ymax):
    """
    Given an infinite line defined by a point p0 and direction d,
    compute its two intersection points with the rectangle defined by:
    x in [xmin, xmax] and y in [ymin, ymax].
    
    Returns:
        (pt1, pt2): the two endpoints of the line segment within the rectangle,
                    or None if the line does not intersect.
    """
    intersections = []
    
    # Intersection with vertical boundaries (x = xmin and x = xmax)
    if d[0] != 0:
        # Left boundary: x = xmin
        t = (xmin - p0[0]) / d[0]
        y = p0[1] + t * d[1]
        if ymin <= y <= ymax:
            intersections.append((xmin, y))
        # Right boundary: x = xmax
        t = (xmax - p0[0]) / d[0]
        y = p0[1] + t * d[1]
        if ymin <= y <= ymax:
            intersections.append((xmax, y))
    
    # Intersection with horizontal boundaries (y = ymin and y = ymax)
    if d[1] != 0:
        # Top boundary: y = ymin
        t = (ymin - p0[1]) / d[1]
        x = p0[0] + t * d[0]
        if xmin <= x <= xmax:
            intersections.append((x, ymin))
        # Bottom boundary: y = ymax
        t = (ymax - p0[1]) / d[1]
        x = p0[0] + t * d[0]
        if xmin <= x <= xmax:
            intersections.append((x, ymax))
    
    # Remove duplicates by converting to a set then back to a list
    intersections = list(set(intersections))
    
    if len(intersections) < 2:
        return None
    else:
        # If there are more than 2, choose the two that are farthest apart.
        # (For a line through a rectangle, you expect exactly 2.)
        if len(intersections) > 2:
            dists = {}
            for i in range(len(intersections)):
                for j in range(i+1, len(intersections)):
                    pt1 = np.array(intersections[i])
                    pt2 = np.array(intersections[j])
                    dists[(i, j)] = np.linalg.norm(pt1 - pt2)
            i, j = max(dists, key=dists.get)
            return intersections[i], intersections[j]
        else:
            return intersections[0], intersections[1]

# --- Process and plot each connected region ---
for i, sl in enumerate(slices, start=1):
    # Extract the region from the original data using its bounding box.
    # Note: the bounding box (sl) may include zero pixels if the region is not rectangular.
    region = vdf_data[sl]
    
    # Create a new figure for this region.
    plt.figure()
    plt.title(f"Region {i} with Parallel Lines (angle = {angle_deg}°)")
    
    # Display the region (using 'gray' colormap).
    # We use origin='upper' so that (0,0) is at the top-left.
    plt.imshow(region, cmap='gray', origin='upper')
    
    # Define the coordinate system for the region slice.
    # x corresponds to columns, y corresponds to rows.
    height, width = region.shape
    xmin, xmax = 0, width
    ymin, ymax = 0, height
    
    # Calculate the center of the bounding box.
    center = np.array([(xmin + xmax) / 2, (ymin + ymax) / 2])
    
    # Determine the range of offsets (along the normal) that cover the rectangle.
    corners = np.array([
        [xmin, ymin],
        [xmax, ymin],
        [xmax, ymax],
        [xmin, ymax]
    ])
    offsets = np.dot(corners - center, n)
    min_offset = np.min(offsets)
    max_offset = np.max(offsets)
    
    # Create a set of offsets (with extra margin) spaced by 'spacing'
    offset_values = np.arange(min_offset - spacing, max_offset + spacing, spacing)
    
    # Create a mask for the nonzero pixels in the region.
    region_mask = (region != 0)
    
    # For each offset, compute and plot the parallel line *only where the region is nonzero*.
    for offset in offset_values:
        # p0 is a point along the normal offset from the center.
        p0 = center + offset * n  
        seg = get_line_segment(p0, d, xmin, xmax, ymin, ymax)
        if seg is None:
            continue
        (x1, y1), (x2, y2) = seg
        
        # Compute the length of the segment and the number of samples
        L = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        num_samples = int(np.ceil(L / sample_step)) + 1
        
        # Parameter t goes from 0 to 1 along the segment.
        t_vals = np.linspace(0, 1, num_samples)
        xs = x1 + t_vals * (x2 - x1)
        ys = y1 + t_vals * (y2 - y1)
        
        # Convert floating-point coordinates to nearest pixel indices.
        # (x corresponds to column, y corresponds to row)
        ix = np.rint(xs).astype(int)
        iy = np.rint(ys).astype(int)
        
        # Ensure indices are within the region boundaries.
        valid = (ix >= 0) & (ix < width) & (iy >= 0) & (iy < height)
        
        # Create an array indicating which sample points fall on a nonzero pixel.
        in_region = np.zeros_like(xs, dtype=bool)
        in_region[valid] = region_mask[iy[valid], ix[valid]]
        
        # Find contiguous intervals in the in_region boolean array.
        intervals = []
        in_interval = False
        start_idx = None
        for j, val in enumerate(in_region):
            if val and not in_interval:
                in_interval = True
                start_idx = j
            elif not val and in_interval:
                intervals.append((start_idx, j - 1))
                in_interval = False
        if in_interval:
            intervals.append((start_idx, len(in_region) - 1))
        
        # For each contiguous interval of points that lie in the nonzero region, plot a line.
        for (start_idx, end_idx) in intervals:
            # Only plot if the interval spans more than one sample.
            if end_idx - start_idx >= 1:
                x_start, y_start = xs[start_idx], ys[start_idx]
                x_end, y_end = xs[end_idx], ys[end_idx]
                plt.plot([x_start, x_end], [y_start, y_end], color='red', linewidth=1)
    
    # Adjust the plot limits and invert the y-axis to match image coordinates.
    plt.xlim(xmin, xmax)
    plt.ylim(ymax, ymin)
    plt.xlabel("X (pixels)")
    plt.ylabel("Y (pixels)")
    plt.show()


In [13]:
import os

# Check if the file containing image_num exists
if os.path.exists("image_num.txt"):
    # If it exists, read the value from the file
    with open("image_num.txt", "r") as file:
        image_num = int(file.read())
else:
    # If it doesn't exist, initialize image_num to 0
    image_num = 0

# Increment image_num
image_num += 1

# Write the updated value back to the file
with open("image_num.txt", "w") as file:
    file.write(str(image_num))

# Print the updated value of image_num
print('current image number',image_num)

current image number 1


In [14]:
#image_num=1#if image imagenum not right modify here
image_num

1

In [15]:
# Check if the dictionaries do not exist in the global scope and create them if necessary
if 'manual_measured_size' not in globals():
    manual_measured_size = {}

if 'semi_manual_measured_size' not in globals():
    semi_manual_measured_size = {}
    
if 'coord_in_SI' not in locals() or 'dict_for_image_storage' not in globals():
    coord_in_SI=[]
coord_in_SI.append(SI_image_coord) # Store the picked spot from SI image.

# Check if dict_for_image_storage exists, if not, initialize it as an empty dictionary
if 'dict_for_image_storage' not in locals() or 'dict_for_image_storage' not in globals():
    dict_for_image_storage = {}
VDF_file_name = '1-{}-VDF'.format(sub_folder)
DP_file_name = '1-{}-DP'.format(sub_folder)
    
# Check if '1-2-VDF' and '1-2-DP' keys exist in dict_for_image_storage
if VDF_file_name not in dict_for_image_storage:
    dict_for_image_storage[VDF_file_name] = {}
if DP_file_name not in dict_for_image_storage:
    dict_for_image_storage[DP_file_name] = {}

# Assuming vdf_data, DP_image_data, SI_image_coord, Center_of_reflection_in_pixel, and radius_in_pixel are defined elsewhere
# Update data in the dictionary
dict_for_image_storage[VDF_file_name][image_num] = (vdf_data, vdf_pixel_to_nm)  # (image#, vdf data)
dict_for_image_storage[DP_file_name][image_num] = (DP_image_data, DP_pixel_to_nm, SI_image_coord, Center_of_reflection_in_pixel, radius_in_pixel)

# Save images in text files
np.savetxt('1-{}-{}-VDF.txt'.format(sub_folder,image_num), dict_for_image_storage[VDF_file_name][image_num][0])  # Save VDF data in a text file
np.savetxt('1-{}-{}-DP.txt'.format(sub_folder,image_num), dict_for_image_storage[DP_file_name][image_num][0])  # Save DP image data in a text file                        


In [16]:
from math import *

# Assuming dict_for_image_storage, image_num, vdf_pixel_to_nm, DP_pixel_to_nm, vdf_data, and DP_image_data are defined

# Plot VDF_data as a grayscale image without a scale bar
vdf_data_with_scale = dict_for_image_storage[VDF_file_name][image_num][0]
vdf_scale_factor = vdf_pixel_to_nm  # Adjust the scale based on vdf_pixel_to_nm
plt.figure(figsize=(8, 6))
plt.imshow(vdf_data_with_scale, cmap='gray', extent=[0, vdf_data_with_scale.shape[1] * vdf_scale_factor, 0, vdf_data_with_scale.shape[0] * vdf_scale_factor])
plt.title('VDF Data (Image {})'.format(image_num))
plt.xlabel('X (nm)')  # X-axis label indicating nanometers
plt.ylabel('Y (nm)')  # Y-axis label indicating nanometers
plt.savefig('1-{}-{}-VDF.tif'.format(sub_folder,image_num))

# Plot DP_image_data as a grayscale image with a scale bar (10/nm) using DP_pixel_to_nm
DP_data_with_scale = dict_for_image_storage[DP_file_name ][image_num][0]
DP_scale_factor = DP_pixel_to_nm  # Adjust the scale based on DP_pixel_to_nm
plt.figure(figsize=(8, 6))
plt.imshow(DP_data_with_scale, cmap='gray', extent=[0, DP_data_with_scale.shape[1] * DP_scale_factor, 0, DP_data_with_scale.shape[0] * DP_scale_factor], vmin=0, vmax=1000)  # Assuming 0-1000 range for grayscale
plt.title('DP Image Data (Image {})'.format(image_num))
plt.xlabel('X (/nm)')  # X-axis label indicating nanometers
plt.ylabel('Y (/nm)')  # Y-axis label indicating nanometers


def deg2rad(deg):
    return deg * pi / 180


image_width, image_height = DP_data_with_scale.shape[1]*s.axes_manager[3].scale, DP_data_with_scale.shape[0]*s.axes_manager[3].scale

circle_points = 100
radius_in_nm = radius # Radius in nanometers
center_x,center_y= Center_of_reflection
center_x += image_width / 2
center_y =image_width / 2-center_y


for angle in range(0, 360, 360 // circle_points):
    x = center_x - radius_in_nm * cos(deg2rad(angle))
    y = center_y + radius_in_nm * sin(deg2rad(angle))
    plt.plot(x, y, marker='o', markersize=1, color='blue') 

plt.show()
plt.savefig('1-{}-{}-DP.tif'.format(sub_folder,image_num))

In [17]:
#Measure the linewidth mannually
from matplotlib.widgets import SpanSelector
if 'manual_coord' not in globals():
    manual_coord= {}

# Assuming dict_for_image_storage, image_num, vdf_pixel_to_nm, and vdf_data are defined
vdf_data = dict_for_image_storage[VDF_file_name ][image_num][0]
vdf_scale_factor = vdf_pixel_to_nm  # Adjust the scale based on vdf_pixel_to_nm

# Lists to store line coordinates and measured lengths
line_coords = []
line_lengths = []

# Function to be called when a line is selected
def onselect(xmin, xmax):
    x_coords = [xmin, xmax]
    line_coords.append(x_coords)
    length_nm = (xmax - xmin)
    line_lengths.append(length_nm)
    print(f'Measured Length: {length_nm:.2f} nm')
    plt.axhline(y=0, color='red', linewidth=2)  # Draw a red line after each measurement
    plt.draw()

fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(vdf_data, cmap='gray', extent=[0, vdf_data.shape[1] * vdf_scale_factor, 0, vdf_data.shape[0] * vdf_scale_factor])
plt.title('VDF Data (Image {})'.format(image_num))
plt.xlabel('X (nm)')  # X-axis label indicating nanometers
plt.ylabel('Y (nm)')  # Y-axis label indicating nanometers

# SpanSelector to draw lines interactively
span = SpanSelector(ax, onselect, 'horizontal', useblit=True, rectprops=dict(alpha=0.5, facecolor='red'))
color_list=['red','green','blue','yellow','white']
plt.show()

# Print the final adjusted lengths
print('Final Adjusted Lengths:')
for i, length in enumerate(line_lengths):
    print(f'Line {i + 1}: {length:.2f} nm')

TypeError: SpanSelector.__init__() got an unexpected keyword argument 'rectprops'

In [18]:
#run the following script to save the figure with measurements and the results
fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(vdf_data, cmap='gray', extent=[0, vdf_data.shape[1] * vdf_scale_factor, 0, vdf_data.shape[0] * vdf_scale_factor])
plt.title('VDF Data (Image {})'.format(image_num))
plt.xlabel('X (nm)')  # X-axis label indicating nanometers
plt.ylabel('Y (nm)')
color_list=['red','green','blue','yellow','white']
plot_position=[-30,vdf_data.shape[0]* vdf_scale_factor+20]
manual_coord[image_num]=np.array([line_coords])
for i in range(len(line_coords)):
    plt.axvline(x=line_coords[i][0], color=color_list[i], linestyle='--', linewidth=2)
    plt.axvline(x=line_coords[i][1], color=color_list[i], linestyle='--', linewidth=2)
    particle_num=i+1
    plt.text(line_coords[i][0], plot_position[i%2],'particle %s \n %s nm'%(particle_num,round(line_lengths[i],1)), fontsize=10, ha='center', va='center',c=color_list[i])
plt.show()
plt.savefig('1-{}-{}-VDF-{}nm_manualy.tif'.format(sub_folder,image_num, line_lengths))
line_lengths_array = np.array(line_lengths)
line_lengths_round=np.round(line_lengths_array,1)
np.savetxt('1-{}-{}-VDF-{}nm-mannauly.txt'.format(sub_folder,image_num, line_lengths_round),line_lengths_round)
manual_measured_size[image_num]=line_lengths

In [19]:
print('mannual_measured_size:',manual_measured_size,'\nmannual_coordination:',manual_coord)

mannual_measured_size: {1: []} 
mannual_coordination: {1: array([], shape=(1, 0), dtype=float64)}


In [20]:
#Semi-manual


# Define Region of Interest (ROI)
x_prime=7.5 #your input
y_prime=17. #your input
import numpy as np
import matplotlib.pyplot as plt
if 'semi_manual_coord' not in globals():
    semi_manual_coord= {}

vdf_data_with_scale = dict_for_image_storage[VDF_file_name][image_num][0]
vdf_scale_factor = vdf_pixel_to_nm  # Adjust the scale based on vdf_pixel_to_nm
plt.figure(figsize=(6, 6))
plt.imshow(vdf_data_with_scale, cmap='gray', extent=[0, vdf_data_with_scale.shape[1] * vdf_scale_factor, 0, vdf_data_with_scale.shape[0] * vdf_scale_factor])
plt.title('VDF Data (Image {})'.format(image_num))
plt.xlabel('X (nm)')  # X-axis label indicating nanometers
plt.ylabel('Y (nm)')  # Y-axis label indicating nanometers


# Define Region of Interest (ROI)
x_prime=7 #your input
y_prime=16. #your input

number_start=round(x_prime/vdf_scale_factor)
number_end=round(y_prime/vdf_scale_factor)

threshold = 30000  # Threshold value for the signal


roi_data = vdf_data[number_start:number_end,:]  # Extract ROI

mid= round((y_prime-x_prime)/2/vdf_scale_factor)
start_col=[]
end_col = []
start_Found=False

for col in range(roi_data.shape[1]):
    if roi_data[mid,col]>threshold and start_Found is False:
        start_col.append(col)
        start_Found=True
    if roi_data[mid,col]<threshold and start_Found:
        end_col.append(col)
        start_Found=False
if len(start_col)==(len(end_col)+1): #if start_col is larger than end_col
    start_col.pop() #remove the last added one

combined_cols = list(zip(start_col, end_col))

combined_cols_array = np.array(combined_cols)
semi_manual_coord[image_num]=combined_cols_array

# Plot the VDF data
plt.imshow(vdf_data, cmap='gray', extent=[0, vdf_data.shape[1], 0, vdf_data.shape[0]])
plt.title('VDF Data (Image {})'.format(image_num))
plt.xlabel('X (pixels)')
plt.ylabel('Y (pixels)')

# Draw the line on the plot
color_list=['red','green','blue','yellow','white']
plot_position=[-30,vdf_data.shape[0]* vdf_scale_factor+20]
for i in range(len(start_col)):
    if start_col is not None and end_col is not None:
        plt.axvline(x=start_col[i], color=color_list[i], linestyle='--', linewidth=2)
        plt.axvline(x=end_col[i], color=color_list[i], linestyle='--', linewidth=2)

plt.show()
start_col_array=np.array(start_col)
end_col_array=np.array(end_col)
measured_length = (end_col_array - start_col_array) * s.axes_manager[1].scale
print('Measured Length:', measured_length, 'nm')

# Save the plot
plt.savefig('1-{}-{}-VDF-{}nm_semi_mannualy.tif'.format(sub_folder,image_num, np.round(measured_length,1)))
np.savetxt('1-{}-{}-VDF-{}nm-semi_mannauly.txt'.format(sub_folder,image_num, np.round(measured_length,1)),measured_length)
semi_manual_measured_size[image_num]=measured_length


NotImplementedError: multi-dimensional slicing is not implemented

In [21]:
print('semi_mannual_measured_size:',semi_mannual_measured_size,'\nsemi_mannual_coordination:',semi_manual_coord)

NameError: name 'semi_mannual_measured_size' is not defined

In [22]:
#export the measured sizes from mannual or semi-mannual
list_semi_mannual_measured_size=[]
for i in range(len(semi_mannual_measured_size)):
    list_semi_mannual_measured_size.append(semi_mannual_measured_size[i+1])

def flatten(data):
    flat_list = []
    for item in data:
        if isinstance(item, (int, float)):
            flat_list.append(item)
        elif isinstance(item, list):
            flat_list.extend(flatten(item))
        elif isinstance(item, np.ndarray):
            flat_list.extend(item.tolist())
    return flat_list

flat_list_semi_mannual_measured_size = flatten(list_semi_mannual_measured_size)

np.savetxt('Total semi_mannual_measured_size.txt',flat_list_semi_mannual_measured_size )

list_mannual_measured_size=[]

for i in range(len(mannual_measured_size)):
    list_mannual_measured_size.append(mannual_measured_size[i+1])


flat_list_mannual_measured_size = flatten(list_mannual_measured_size)
np.savetxt('Total mannual_measured_size.txt',flat_list_mannual_measured_size )


NameError: name 'semi_mannual_measured_size' is not defined

In [23]:
#Save all the files # dicts
import pickle


dicts_to_save = {
    'mannual_measured_size': manual_measured_size,
    'semi_mannual_measured_size': semi_manual_measured_size,
    'dict_for_image_storage': dict_for_image_storage,
    'semi_manual_coord':semi_manual_coord,
    'manual_coord': manual_coord
}

with open('saved_dicts.pkl', 'wb') as file:
    pickle.dump(dicts_to_save, file)

TypeError: cannot pickle memoryview objects

In [24]:
# Load the dictionary containing our data
with open('saved_dicts.pkl', 'rb') as file:
    loaded_dicts = pickle.load(file)

# Access individual dictionaries using their variable names
#mannual_measured_size = loaded_dicts['mannual_measured_size']
#semi_mannual_measured_size = loaded_dicts['semi_mannual_measured_size']
#dict_for_image_storage = loaded_dicts['dict_for_image_storage']


EOFError: Ran out of input

In [25]:
#After all the measurements done. Plotting all the VDF images
num_images = len(dict_for_image_storage[VDF_file_name])
fig, axes = plt.subplots(num_images, 1, figsize=(6, 6*num_images))
semi_mannual_measured_size_round = {k: round(v, 1) if not isinstance(v, np.ndarray) else np.round(v, 1) for k, v in semi_manual_measured_size.items()}
mannual_measured_size_round = {k: [round(num, 1) for num in v] for k, v in manual_measured_size.items()}
# Iterate through the dictionary and plot images vertically
for i in range(1, num_images + 1):
    data, scale_factor = dict_for_image_storage[VDF_file_name][i]
    ax = axes[i - 1]  # Select the appropriate subplot
    ax.imshow(data, cmap='gray', extent=[0, data.shape[1] * scale_factor, 0, data.shape[0] * scale_factor])
    ax.set_title('VDF (image{} from point {} with sizes {}measured mannually)'.format(i, dict_for_image_storage[DP_file_name][i][2], mannual_measured_size_round[i]))
    ax.set_xlabel('X (nm)')  # Adjust the label according to your data unit
    ax.set_ylabel('Y (nm)')  # Adjust the label according to your data unit

plt.tight_layout()  # Adjust spacing between subplots for better visualization
plt.savefig('Size measuremnts.tif') 
plt.show()

TypeError: 'Axes' object is not subscriptable

In [26]:
# Calculate the number of rows and columns for subplots
num_images = len(dict_for_image_storage[DP_file_name])
num_cols = int(np.sqrt(num_images))  # Number of columns (square root of num_images)
num_rows = num_images // num_cols  # Number of rows
image_width=
if num_images % num_cols != 0:
    num_rows += 1
# Create subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 6*num_rows))

# Iterate through the dictionary and plot images horizontally
for i in range(num_images):
    row = i // num_cols  # Calculate the row index
    col = i % num_cols  # Calculate the column index
    
    data = dict_for_image_storage[DP_file_name][i + 1][0]
    scale_factor = dict_for_image_storage[DP_file_name][i + 1][1]
    ax = axes[row, col]  # Select the appropriate subplot
    max_intensity = data.max()  # Calculate the maximum intensity value of the image
    upper_intensity_limit = max_intensity / 10  # Set the upper intensity limit to 1/3 of the maximum intensity

    # Plot the image with adjusted upper intensity limit
    ax.imshow(data, cmap='gray', extent=[0, data.shape[1] * scale_factor, 0, data.shape[0] * scale_factor], vmax=upper_intensity_limit)
    ax.set_title('DP (image {} from point {} with reflection point at {})'.format(i + 1, dict_for_image_storage[DP_file_name][i + 1][2], 
                                                                                  dict_for_image_storage[DP_file_name][i + 1][3]))
    ax.set_xlabel('X (nm)')  # Adjust the label according to your data unit
    ax.set_ylabel('Y (nm)')  # Adjust the label according to your data unit
    circle_points = 100
    radius_in_nm = dict_for_image_storage[DP_file_name][i + 1][4]*scale_factor  # Radius in pixel
    
    center_x, center_y = [coord * scale_factor for coord in dict_for_image_storage[DP_file_name][i + 1][3]]
    center_x += image_width / 2
    center_y = image_width / 2 - center_y
    
    for angle in range(0, 360, 360 // circle_points):
        x = center_x - radius_in_nm * cos(deg2rad(angle))
        y = center_y + radius_in_nm * sin(deg2rad(angle))
        ax.plot(x, y, marker='o', markersize=1, color='blue')

plt.tight_layout()  # Adjust spacing between subplots for better visualization
plt.savefig('All_DPs.tif')  # Save the plot as an image file
plt.show()

SyntaxError: invalid syntax (2980653294.py, line 5)

In [27]:
len(dict_for_image_storage[DP_file_name])

1